# **Import**

---



In [12]:
import numpy as np 
import os
import shutil
import math
import cv2
import tensorflow as tf
import json 
import matplotlib.pyplot as plt
import pandas as pd
import random as rd
import pickle as pk

import time

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Lambda

# load the pretrained network from Keras Applications
from keras_preprocessing.image import load_img
from keras_preprocessing.image import img_to_array
from keras.applications.vgg16 import VGG16, preprocess_input #224*224
from keras.applications.xception import Xception, preprocess_input, decode_predictions #299*299
from keras.applications.mobilenet import MobileNet, preprocess_input, decode_predictions #224*224

from tqdm import tqdm

# **Database download**

---



In [ ]:
!rm -rf MIR_DATASETS_B
!wget https://github.com/sidimahmoudi/facenet_tf2/releases/download/AI_MIR_CLOUD/MIR_DATASETS_B.zip 
!unzip -q /content/MIR_DATASETS_B.zip
!rm MIR_DATASETS_B.zip

# **Database preprocess**

---



In [3]:
for animal in os.listdir('MIR_DATASETS_B'):
  for type_animal in os.listdir(os.path.join('MIR_DATASETS_B', animal)):
    for file in os.listdir(os.path.join('MIR_DATASETS_B', animal, type_animal)):
      shutil.copy2(os.path.join('MIR_DATASETS_B', animal, type_animal, file), os.path.join('MIR_DATASETS_B'))

# **RMAC Layer**

---



In [4]:
# from  https://github.com/v-v/RMAC-TensorFlow-2

class RMAC:
    def __init__(self, shape, levels=3, power=None, overlap=0.4, norm_fm=False, sum_fm=True, verbose=False):
        self.shape = shape
        self.sum_fm = sum_fm
        self.norm = norm_fm
        self.power = power

        # ported from Giorgios' Matlab code
        steps = np.asarray([2, 3, 4, 5, 6, 7])
        B, H, W, D = shape
        w = min([W, H])
        w2 = w // 2 - 1
        b = np.asarray((max(H, W) - w)) / (steps - 1);
        idx = np.argmin(np.abs(((w**2 - w*b)/(w**2))-overlap))

        Wd = 0
        Hd = 0
        if H < W:
            Wd = idx + 1
        elif H > W:
            Hd = idx + 1

        self.regions = []
        for l in range(levels):

            wl = int(2 * w/(l+2));
            wl2 = int(wl / 2 - 1);

            b = 0 if not (l + Wd) else ((W - wl) / (l + Wd))
            cenW = np.asarray(np.floor(wl2 + np.asarray(range(l+Wd+1)) * b), dtype=np.int32) - wl2
            b = 0 if not (l + Hd) else ((H - wl) / (l + Hd))
            cenH = np.asarray(np.floor(wl2 + np.asarray(range(l+Hd+1)) * b), dtype=np.int32) - wl2

            for i in cenH:
                for j in cenW:
                    if i >= W or j >= H:
                        continue
                    ie = i+wl
                    je = j+wl
                    if ie >= W:
                        ie = W
                    if je >= H:
                        je = H
                    if ie - i < 1 or je - j < 1:
                        continue
                    self.regions.append((i,j,ie,je))

        if verbose:
            print('RMAC regions = %s' % self.regions)

    def rmac(self, x):
        y = []
        for r in self.regions:
            x_sliced = x[:, r[1]:r[3], r[0]:r[2], :]
            if self.power is None:
                x_maxed = tf.reduce_max(x_sliced, axis=(1,2))
            else:
                x_maxed = tf.reduce_mean((x_sliced ** self.power), axis=(2,3)) ** (1.0 / self.power)
                x_maxed = tf.pow(tf.reduce_mean((tf.pow(x_sliced, self.power)), axis=(2,3)),(1.0 / self.power))
            y.append(x_maxed)

        y = tf.stack(y, axis=0)
        y = tf.transpose(y, [1,0,2])

        if self.norm:
            y = tf.math.l2_normalize(y, 2)


        if self.sum_fm:
            y = tf.reduce_mean(y, axis=(1))

        return y

# **RMAC indexation**

---



In [16]:
def rmac(folder, base_model, layer, input_dim):
    
    data = list()

    base_out = base_model.get_layer(layer).output
    rmac = RMAC(base_out.shape, levels=5, norm_fm=True, sum_fm=True)    
    rmac_layer = Lambda(rmac.rmac, input_shape=base_model.output_shape, name="rmac_"+layer)
    out = rmac_layer(base_out)
    model = Model(base_model.input, out)

    for path in tqdm(os.listdir(folder)):
      if '.jpg' in path:
        filename = os.path.join(folder, path)
        image = load_img(filename, target_size=(input_dim, input_dim))
        image = img_to_array(image)
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        image = preprocess_input(image)
        if image is not None:
            feature = model.predict(image)
            feature = np.array(feature[0])
            num_image, _ = path.split(".")
            data.append({num_image : feature})
    return data

def save(data, file_name):
    with open(file_name, 'w') as f:
        json.dump(data, f, cls = JsonCustomEncoder)

class JsonCustomEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.ndarray, np.number)):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [ ]:
base_model = Xception(include_top = False, weights ='imagenet', input_shape = (299, 299, 3), pooling = 'avg')
data = rmac('MIR_DATASETS_B/', base_model, 'block14_sepconv2', 299)
save(data, 'XCEPTION_false_rmac.json')

base_model = Xception(include_top = True, weights ='imagenet', input_shape = (299, 299, 3), pooling = 'avg')
data = rmac('MIR_DATASETS_B/', base_model, 'block14_sepconv2', 299)
save(data, 'XCEPTION_true_rmac.json')

base_model = VGG16(include_top = False, weights ='imagenet', input_shape = (224, 224, 3), pooling = 'avg')
data = rmac('MIR_DATASETS_B/', base_model, 'block5_conv3', 224)
save(data, 'VGG16_false_rmac.json')

base_model = MobileNet(include_top = False, weights ='imagenet', input_shape = (224, 224, 3), pooling = 'avg')
data = rmac('MIR_DATASETS_B/', base_model, 'conv_pw_13', 224)
save(data, 'MOBILENET_false_rmac.json')